# Thalamus segments in MNI space

> Import libraries

In [31]:
import pandas as pd
import numpy as np
import os
from os.path import join
from os import listdir

import nibabel as nb

from multiprocessing import Pool
import itertools

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

> Set variables

In [ ]:
server = '147.47.238.209'
dataLoc = '/Volume/CCNC_W1_2T/2017_CHR_thalamus_microstructure_kcho/allData'
subjects = [x for x in listdir(dataLoc) if x.startswith('CHR') or x.startswith('NOR')]

> Thalamus segmentation data example

In [30]:
pd.DataFrame(np.random.randint(0, 9, 36).reshape(6,6),
            index=['voxel_x_{0}'.format(x) for x in np.arange(6)],
            columns=['voxel_y_{0}'.format(x) for x in np.arange(6)])

,voxel_y_0,voxel_y_1,voxel_y_2,voxel_y_3,voxel_y_4,voxel_y_5
voxel_x_0,7,3,6,5,5,4
voxel_x_1,5,7,1,6,6,3
voxel_x_2,4,8,4,6,2,8
voxel_x_3,2,6,8,2,3,1
voxel_x_4,7,0,5,2,1,8
voxel_x_5,8,5,1,2,6,6


> Make class

In [9]:
class thalamus:
    def __init__(self, subjectLoc, side):
        mni_biggest_loc = join(subjectLoc, '{side}_biggest_mni.nii.gz')
        self.data = nb.load(mni_biggest_loc).get_data()

> Reading subject map as matrix

In [ ]:
s = thalamus(join(dataLoc, subjects[0]), 'lh')

In [ ]:
s.data

In [ ]:
plt.imshow(s.data[:,:,60])

> Reading all subject maps

In [32]:
pool = Pool()

# Make inputs
subjectLocs = [join(dataLoc, x) for x in subjects]

a = [subjectLocs, ['lh']]
all_combinations = list(itertools.product(*a))
left_outputs = pool.map(thalamus, all_combinations)

a = [subjectLocs, ['rh']]
all_combinations = list(itertools.product(*a))
right_outputs = pool.map(thalamus, all_combinations)

NameError: name 'subjects' is not defined

> Merge the 3D matrices into one 4D matrix

In [ ]:
allData_left = np.concatenate([x.data[:,:,:,np.newaxis] for x in left_outputs])
allData_right = np.concatenate([x.data[:,:,:,np.newaxis] for x in right_outputs])

# Assigning most occuring value to the voxel

http://stackoverflow.com/questions/12297016/how-to-find-most-frequent-values-in-numpy-ndarray

> Left

In [ ]:
u, indices= np.unique(allData_left, return_inverse=True)
axis = 3
biggest_left = u[np.argmax(np.apply_along_axis(np.bincount, 
                                               axis, 
                                               indices.reshape(allData_left.shape),
                                               None, 
                                               p.max(indices) + 1), 
                      axis=axis)]

> Right

In [ ]:
u, indices= np.unique(allData_right, return_inverse=True)
axis = 3
biggest_right = u[np.argmax(np.apply_along_axis(np.bincount, 
                                          axis, 
                                          indices.reshape(allData_right.shape),
                                          None, 
                                          p.max(indices) + 1), 
                      axis=axis)]

# Merge left and right

In [ ]:
biggest = biggest_left + biggest_right

# Data visualization

In [ ]:
biggest_right.shape

In [ ]:
plt.imshow(biggest[:,:,])